### 1.Sa se scrie un program pentru implementarea algoritmului de analiza sintactica Earley. Programul primeste la intrare elementele unei gramatici independente de context oarecare, inclusiv cu 𝜆-productii. Programul accepta un numar oarecare de siruri peste alfabetul terminalilor. Pentru fiecare sir se creeaza si se afiseaza tabelele Earley corespondente si daca sirul apartine limbajului generat de gramatica, afiseaza derivarile acelui sir plecand din simbolul de start.

In [29]:
def predictor(rule, state):
    if rule["right"][rule["dot"]].isupper(): # NON-TERMINAL
        return [{
            "left": rule["right"][rule["dot"]],
            "right": right,
            "dot": 0,
            "op": "PREDICTOR",
            "completer": []
        } for right in rules[rule["right"][rule["dot"]]]] 
    else:
        return []


def scanner(rule, next_input):
    # TERMINAL
    if rule["right"][rule["dot"]].islower() and next_input in rules[rule["right"][rule["dot"]]]:
        return [{
            "left": rule["right"][rule["dot"]],
            "right": [next_input],
            "dot": 1,
            "op": "SCANNER",
            "completer": []
        }] 
    else:
        return []

def completer(rule, charts):
    if rule["dot"] == len(rule["right"]):
        return list(map(
            lambda filter_rule: {
                "left": filter_rule["left"],
                "right": filter_rule["right"],
                "dot": filter_rule["dot"] + 1,
                "op": "COMPLETER",
                "completer": [rule] + filter_rule["completer"]
            },
            filter(
                lambda p_rule: p_rule["dot"] < len(p_rule["right"]) and rule["left"] == p_rule["right"][p_rule["dot"]],
                charts[rule["state"]]
            )
        )) 
    else:
        return []

In [30]:
## input
N = ['S', 'X'] # NON-TERMINALS
T = ['a', 'b', ''] # TERMINALS
S = 'S' # START SYMBOL
rules = {
    "S": [
        ['aXbX'],
        ['bXaX'],
    ],
    "X" : [
        ['aXbX'],
        ['bXaX'],
        ['']
    ]
}
input_string = 'abba'
input_arr = [char for char in input_string] + ['']
charts = [[{
    "left": "S'",
    "right": ["S"],
    "dot": 0,
    "op": "EXTEND",
    "completer": []
}]]

for curr_state in range(len(input_arr)):
    curr_chart = charts[curr_state]
    next_chart = []

    for curr_rule in curr_chart:
        for element in curr_rule['right']:
            if curr_rule["dot"] < len(element): # not finished
                curr_chart += [i for i in predictor(curr_rule, curr_state) if i not in curr_chart] # PREDICTOR
                next_chart += [i for i in scanner(curr_rule, input_arr[curr_state]) if i not in next_chart] # SCANNER
            else:
                curr_chart += [i for i in completer(curr_rule, charts) if i not in curr_chart] # COMPLETOR

    charts.append(next_chart)

In [31]:
def print_charts(charts, inp):
    for chart_no, chart in zip(range(len(charts)), charts):
        print("\t{}".format("S" + str(chart_no)))
        print("\t\n".join(map(
            lambda x: "\t{} --> {}, {} {}".format(
                x["left"], 
                "".join(x["right"][:x["dot"]] + ["."] + x["right"][x["dot"]:]),
                str(chart_no) + ',',
                x["op"]
            ),
            chart
        )))
        print()
print_charts(charts[:-1], input_arr)

	S0
	S' --> .S, 0, EXTEND	
	S --> .aXbX, 0, PREDICTOR	
	S --> .bXaX, 0, PREDICTOR

	S1


	S2


	S3


	S4


